<a href="https://colab.research.google.com/github/mmendezs2021/Em-prende/blob/main/DataExploration_preliminar_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (12,12)

In [2]:
import numpy as np # linear algebra
from sklearn.preprocessing import LabelEncoder
import re

In [3]:
!pip install PyDrive

In [4]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [5]:
auth.authenticate_user()

In [6]:
gauth = GoogleAuth()

In [7]:
gauth.credentials = GoogleCredentials.get_application_default()

In [8]:
drive = GoogleDrive(gauth)

In [9]:
download = drive.CreateFile({'id': '13wq6Z0DWPHRPIZdiyCCmjdvqCp2CKgy4'})
download.GetContentFile('Set_28Jul_3PM_sin_missingvalues_todonormalizado.csv')

In [10]:
# read dataset
#sep=None Python parsing engine will be used and automatically
# detect the separator by Python’s builtin sniffer tool
train_df = pd.read_csv('./Set_28Jul_3PM_sin_missingvalues_todonormalizado.csv',sep=None,skiprows=0)
#                       ,dtype={"pc_1":np.float64,
#"Probabilidad de viabilidad":str,"RUC":str,
#"pc_2":np.float64,"pc_3":np.float64,"pc_4":np.float64,"pc_5":np.float64,"pc_6":np.float64,
#"pc_7":np.float64,"pc_8":np.float64,"pc_9":np.float64,"pc_10":np.float64,"pc_11":np.float64})# global random state

rand_state_ = 42

train_df.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


(35481, 44)

In [ ]:
class DataPreparation():
    def __init__(self):
        pass
        
        
    def preprocess(self, df):
        df = self.fill_missing_values(df)
        df = self.feature_extraction(df)
        df = self.handle_categorical_variables(df)
        df = self.dimensionality_reduction(df)
        return df

In [11]:
train_df.describe

<bound method NDFrame.describe of        ClasificacionEmpresas_Pequeña empresa  ...           RUC
0                                          1  ...  1.790200e+12
1                                          0  ...  1.790040e+12
2                                          0  ...  1.790510e+12
3                                          1  ...  1.391770e+12
4                                          1  ...  9.900260e+11
...                                      ...  ...           ...
35476                                      0  ...  9.924580e+11
35477                                      0  ...  9.924870e+11
35478                                      1  ...  9.925510e+11
35479                                      0  ...  9.924640e+11
35480                                      0  ...  9.914410e+11

[35481 rows x 44 columns]>

In [12]:
train_df.head(10)

,ClasificacionEmpresas_Pequeña empresa,ClasificacionEmpresas_Grande empresa,ClasificacionEmpresas_Micro empresa,ClasificacionEmpresas_Mediana empresa,ACTIVIDAD ECONOMICA_A,ACTIVIDAD ECONOMICA_B,ACTIVIDAD ECONOMICA_C,ACTIVIDAD ECONOMICA_D,ACTIVIDAD ECONOMICA_E,ACTIVIDAD ECONOMICA_F,ACTIVIDAD ECONOMICA_G,ACTIVIDAD ECONOMICA_H,ACTIVIDAD ECONOMICA_I,ACTIVIDAD ECONOMICA_J,ACTIVIDAD ECONOMICA_K,ACTIVIDAD ECONOMICA_L,ACTIVIDAD ECONOMICA_M,ACTIVIDAD ECONOMICA_N,ACTIVIDAD ECONOMICA_O,ACTIVIDAD ECONOMICA_P,ACTIVIDAD ECONOMICA_Q,ACTIVIDAD ECONOMICA_R,ACTIVIDAD ECONOMICA_S,ACTIVIDAD ECONOMICA_T,ACTIVIDAD ECONOMICA_U,median(TOTAL DEL ACTIVO (399)),median(TOTAL DEL PASIVO (499)),median(TOTAL PATRIMONIO NETO (598)),median(TOTAL PASIVO Y PATRIMONIO (599)),median(TOTAL INGRESOS (699)),median(PERDIDA DEL EJERCICIO (802)),median(15% PART TRABAJADORES (803)),median(VENTAS NETAS LOCALES 12% (601)),median(VENTAS NETAS LOCALES 0% (602)),median(EXPORTACIONES NETAS (603)),median(TOTAL COSTOS (797)),median(TOTAL GASTOS (798)),median(UTILIDAD DEL EJERCICIO (801)),median(UTILIDAD GRAVABLE (819)),median(PERDIDA (829)),median(UTILIDAD A REINV Y CAPITALIZAR (831)),median(TOTAL IMPUESTO CAUSADO (839)),p,RUC
0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.790200e+12
1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.790040e+12
2,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.790510e+12
3,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.391770e+12
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900260e+11
5,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.792240e+12
6,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900060e+11
7,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900090e+11
8,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900090e+11
9,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900090e+11


In [16]:
train2_df = (train_df[0:11]
             .rename(columns={
                "Probabilidad de viabilidad":"p",
            })
            .query("RUC>-1.0")
)
train2_df

,ClasificacionEmpresas_Pequeña empresa,ClasificacionEmpresas_Grande empresa,ClasificacionEmpresas_Micro empresa,ClasificacionEmpresas_Mediana empresa,ACTIVIDAD ECONOMICA_A,ACTIVIDAD ECONOMICA_B,ACTIVIDAD ECONOMICA_C,ACTIVIDAD ECONOMICA_D,ACTIVIDAD ECONOMICA_E,ACTIVIDAD ECONOMICA_F,ACTIVIDAD ECONOMICA_G,ACTIVIDAD ECONOMICA_H,ACTIVIDAD ECONOMICA_I,ACTIVIDAD ECONOMICA_J,ACTIVIDAD ECONOMICA_K,ACTIVIDAD ECONOMICA_L,ACTIVIDAD ECONOMICA_M,ACTIVIDAD ECONOMICA_N,ACTIVIDAD ECONOMICA_O,ACTIVIDAD ECONOMICA_P,ACTIVIDAD ECONOMICA_Q,ACTIVIDAD ECONOMICA_R,ACTIVIDAD ECONOMICA_S,ACTIVIDAD ECONOMICA_T,ACTIVIDAD ECONOMICA_U,median(TOTAL DEL ACTIVO (399)),median(TOTAL DEL PASIVO (499)),median(TOTAL PATRIMONIO NETO (598)),median(TOTAL PASIVO Y PATRIMONIO (599)),median(TOTAL INGRESOS (699)),median(PERDIDA DEL EJERCICIO (802)),median(15% PART TRABAJADORES (803)),median(VENTAS NETAS LOCALES 12% (601)),median(VENTAS NETAS LOCALES 0% (602)),median(EXPORTACIONES NETAS (603)),median(TOTAL COSTOS (797)),median(TOTAL GASTOS (798)),median(UTILIDAD DEL EJERCICIO (801)),median(UTILIDAD GRAVABLE (819)),median(PERDIDA (829)),median(UTILIDAD A REINV Y CAPITALIZAR (831)),median(TOTAL IMPUESTO CAUSADO (839)),p,RUC
0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.790200e+12
1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.790040e+12
2,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.790510e+12
3,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.391770e+12
4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900260e+11
5,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,1.792240e+12
6,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900060e+11
7,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900090e+11
8,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900090e+11
9,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.173042,-0.168286,-0.10378,-0.173042,-0.315766,-0.274855,-0.196039,-0.275695,-0.261492,-0.03605,-0.233495,-0.309722,-0.168209,-0.191369,-0.333656,2.179907,-0.189258,alta,9.900090e+11


In [ ]:
train3_df=pd.DataFrame(train2_df,columns=['pc_1','pc_2'],dtype=float)
#print train3_df

In [ ]:
train3_df.describe

<bound method NDFrame.describe of            pc_1      pc_2
0     -0.805826  0.101677
1     -0.805826  0.101677
2     -0.805826  0.101677
3     -0.805826  0.101677
4     -0.805826  0.101677
...         ...       ...
28380 -0.813247  0.311413
28381 -0.813247  0.311413
28382 -0.813247  0.311413
28383 -0.577915  0.114637
28384 -0.607033  0.050559

[28385 rows x 2 columns]>

In [ ]:
train2_df.columns

Index(['p', 'RUC', 'pc_1', 'pc_2', 'pc_3', 'pc_4', 'pc_5', 'pc_6', 'pc_7',
       'pc_8', 'pc_9', 'pc_10', 'pc_11'],
      dtype='object')

In [ ]:
train2_df.count

<bound method DataFrame.count of                p         RUC      pc_1  ...      pc_9     pc_10     pc_11
0           alta   1.7902E12 -0.805826  ...  0.041961 -0.023338  0.487120
1           alta  1.79004E12 -0.805826  ...  0.041961 -0.023338  0.487120
2           alta  1.79051E12 -0.805826  ...  0.041961 -0.023338  0.487120
3           alta  1.39177E12 -0.805826  ...  0.041961 -0.023338  0.487120
4           alta  9.90026E11 -0.805826  ...  0.041961 -0.023338  0.487120
...          ...         ...       ...  ...       ...       ...       ...
28380  Mediabaja  9.92429E11 -0.813247  ... -0.005897  0.032181 -0.655407
28381  Mediabaja  9.92458E11 -0.813247  ... -0.005897  0.032181 -0.655407
28382  Mediabaja  9.92487E11 -0.813247  ... -0.005897  0.032181 -0.655407
28383  Mediabaja  9.92464E11 -0.577915  ...  0.045695  0.135463 -0.645764
28384  Mediabaja  9.91441E11 -0.607033  ... -0.016848 -0.063184 -0.776117

[28385 rows x 13 columns]>

In [ ]:
train2_df.dtypes

p         object
RUC       object
pc_1     float64
pc_2     float64
pc_3     float64
pc_4     float64
pc_5     float64
pc_6     float64
pc_7     float64
pc_8     float64
pc_9     float64
pc_10    float64
pc_11    float64
dtype: object

Finalmente, guardamos a csv el dataset limpio

In [17]:
train2_df.to_csv("./test4Aug.csv", index=False)